In [1]:
# 图太大，直接durchaverage估计不行，应该试试别的方法

In [2]:
from collections import namedtuple
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [10]:
tr = pd.read_pickle(traindata)

In [11]:
tr.head()

,bounding box,Jobid,Rot,score
0,"(0.0, 0.0, 202.0, 87.0)",35126-0_1,90.00,3.629176
1,"(0.0, 0.0, 225.0, 162.0)",35126-0_1,90.00,1.233175
2,"(0.0, 0.0, 608.5, 117.3)",35126-0_1,90.00,1.151725
3,"(0.0, 0.0, 265.0, 142.0)",35126-0_1,90.00,0.644703
4,"(0.0, 0.0, 170.0, 154.0)",35126-0_1,0.00,1.164499


In [8]:
# run the model
# use one GPU
traindata = '../data/train.pickle'
polysdata = '../data/matrix_feature.pickle'
# set hype
num_epochs = 200
batch_size = 2 # because of the suqeeze function the batch_size should not be one
verbose = True
# set models and loss
model = CnnLSTM()
if torch.cuda.is_available():
    model = model.cuda()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
# set the scheduler
#lamb1 = lambda x: .1**(x//30)
#scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda = lamb1)
# transform data
train = Data(traindata, polysdata)
test = Data(traindata, polysdata)
dl_train = DataLoader(train, batch_size = batch_size, shuffle = True)
dl_test = DataLoader(test, batch_size = batch_size, shuffle = True)

In [25]:
# train the model
for epoch in range(num_epochs):
    #scheduler.step()
    model.train()
    train_loss = 0
    counter = 0
    for batch_idx, dat in enumerate(dl_train):
        counter += 1
        # train the model
        optimizer.zero_grad()
        inp, target = dat
        out = model(inp)
        lo = loss(out.squeeze(), target.squeeze())
        lo.backward()
        optimizer.step()
        train_loss += lo.data
        if verbose:
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * batch_size,
                    len(train),
                    100.*batch_idx*batch_size/len(train),
                    lo.data
                    ))
    val_lo = val(dl_test, model, loss)
    if num_epochs % 10 == 1:
        # save model pro 100 rounds
        torch.save(model.state_dict(), '../models/incpLSTM'+'_'+str(num_epochs/10))
    num_epochs += 1
    # test_lo = test(dl_test, model, loss)
    if verbose:
        # train loss
        print('====> Epoch: {} Average train loss: {:.4f}'.format(
            epoch,
            train_loss/counter
            ))
        # test loss
        print('====> Epoch: {} Average val loss: {:.4f}'.format(
            epoch,
            val_lo
            ))

Train Epoch: 0 [0/4000 (0%)]	Loss: 4.109964
Train Epoch: 0 [20/4000 (0%)]	Loss: 1.461376
Train Epoch: 0 [40/4000 (1%)]	Loss: 0.681544
Train Epoch: 0 [60/4000 (2%)]	Loss: 0.462741
Train Epoch: 0 [80/4000 (2%)]	Loss: 0.870093
Train Epoch: 0 [100/4000 (2%)]	Loss: 0.430903
Train Epoch: 0 [120/4000 (3%)]	Loss: 0.575731
Train Epoch: 0 [140/4000 (4%)]	Loss: 0.449890
Train Epoch: 0 [160/4000 (4%)]	Loss: 0.461239
Train Epoch: 0 [180/4000 (4%)]	Loss: 0.591531
Train Epoch: 0 [200/4000 (5%)]	Loss: 0.473408
Train Epoch: 0 [220/4000 (6%)]	Loss: 0.395688
Train Epoch: 0 [240/4000 (6%)]	Loss: 0.665757
Train Epoch: 0 [260/4000 (6%)]	Loss: 0.539714
Train Epoch: 0 [280/4000 (7%)]	Loss: 0.474875
Train Epoch: 0 [300/4000 (8%)]	Loss: 0.753829
Train Epoch: 0 [320/4000 (8%)]	Loss: 0.446841
Train Epoch: 0 [340/4000 (8%)]	Loss: 0.375991
Train Epoch: 0 [360/4000 (9%)]	Loss: 0.718455
Train Epoch: 0 [380/4000 (10%)]	Loss: 0.604783
Train Epoch: 0 [400/4000 (10%)]	Loss: 0.386382
Train Epoch: 0 [420/4000 (10%)]	Loss: 

KeyboardInterrupt: 

In [5]:
class Data():
    def __init__(self, x, y):
        self.data = {}
        self.data['data'] = self.add_file(x) # type of pandas 
        self.data['polys'] = self.add_file(y) # type of pandas
        self.jobs = self.data['data']['Jobid'].unique()
        self.len = len(self.jobs)
    
    def add_file(self, path):
        # read pickle file with pandas
        out = pd.read_pickle(path)
        out = out.astype({'Rot': 'float'})
        return out.iloc[:200000, :]
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        # get item with jobid
        dat = self.data['data'][self.data['data']['Jobid'] == self.jobs[index]]
        # merge 
        dat = dat.merge(self.data['polys'], on=['bounding box', 'Rot'], how = 'left')[['matrix', 'score']]
        # transform and concatenate and split data 
        dat_x = np.concatenate(dat[['matrix']].values.tolist(), axis = 0) # shape of [50, 350, 350]
        dat_y = dat[['score']].values # shape of [50, 1]
        # transofrom to torch
        dat_x = torch.from_numpy(dat_x).type('torch.FloatTensor')
        dat_y = torch.from_numpy(dat_y).type('torch.FloatTensor')
        # move to gpu if exist
        if torch.cuda.is_available():
            dat_x = dat_x.cuda()
            dat_y = dat_y.cuda()
        return dat_x, dat_y

In [ ]:
class Data2():
    # load data at once
    def __init__(self, x, y):
        self.data = {}
        self.data['data'] = self.add_file(x) # type of pandas 
        self.data['polys'] = self.add_file(y) # type of pandas
        self._combine()
        #self.jobs = self.data['data']['Jobid'].unique()
        self.len = len(self.jobs)
    
    def _combine(self):
        # get item with jobid
        jobs = self.data['data']['Jobid'].unique()
        counter = 0
        Xs = []
        ys = []
        for i in jobs:
            dat = self.data['data'][self.data['data']['Jobid'] == i]
            # merge 
            dat = dat.merge(self.data['polys'], on=['bounding box', 'Rot'], how = 'left')[['matrix', 'score']]
            # transform and concatenate and split data 
            dat_x = np.concatenate(dat[['matrix']].values.tolist(), axis = 0) # shape of [50, 350, 350]
            dat_y = dat[['score']].values # shape of [50, 1]
            # transofrom to torch
            dat_x = torch.from_numpy(dat_x).type('torch.FloatTensor')
            dat_y = torch.from_numpy(dat_y).type('torch.FloatTensor')
            # move to gpu if exist
            if torch.cuda.is_available():
                dat_x = dat_x.cuda()
                dat_y = dat_y.cuda()
            Xs.append(dat_x)
            ys.append(dat_y)
            # add counter 
            counter += 1
        del self.data['data']
        self.data['Xs'] = Xs
        self.data['ys'] = ys
    
    def add_file(self, path):
        # read pickle file with pandas
        out = pd.read_pickle(path)
        out = out.astype({'Rot': 'float'})
        return out.iloc[:200000, :]
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.data['Xs'][index], self.data['ys'][index]

In [6]:
class CnnLSTM(nn.Module):
    def __init__(self, num_classes = 1):
        super(CnnLSTM, self).__init__()
        num_hidden = 128
        self.cnn = nn.Sequential(
            #BasicConv2d(1, 16, kernel_size=3, stride=2),
            BasicConv2d(1, 16, kernel_size=3),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            BasicConv2d(16, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout2d())
        self.lstm = nn.LSTM(32, num_hidden, num_layers = 2, batch_first = True)
        self.fce = nn.Linear(num_hidden, num_classes)
    def forward(self, xs):
        """
        parameters:
        -------------
            xs: input data, list of pictures (with shape (N, 299, 299, 1))
        """
        co = []
        out = []
        seq_len = len(xs)
        batch_size, seq_len, width, height = xs.shape
        for i in range(seq_len):
            # (N, 299, 299, 1) => (N, num_hidden)
            #co.append(self.cnn(torch.flatten(x, 1)))
            #print('%'*20)
            #print('round: '+ str(i))
            tmp = self.cnn(xs[:,i,:,:].unsqueeze(1))
            co.append(tmp.squeeze())
            #print(tmp.squeeze().shape)
        co = [i.unsqueeze(1) for i in co]
        co = torch.cat(co, 1)
        lo, _ = self.lstm(co)
        for i in range(seq_len):
            tmp = self.fce(lo[:, i,:])
            out.append(tmp)
        out = torch.cat(out, 1)
        return out.squeeze()

In [7]:
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [28]:
import time

In [35]:
3%1

0

In [33]:
time.strftime(time.time())

TypeError: strftime() argument 1 must be str, not float